## Generating n home listings 

In [3]:
from pathlib import Path
import json, time
from typing import List
import json, time
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() #load all the env variables

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [12]:
# --- 1. model --------------------------------------------------------------
llm_groq = ChatGroq(
    model_name="gemma2-9b-it",  # or another Groq model
    temperature=0.8,
    max_tokens=512   # plenty for one JSON listing
)

llm_groq.invoke("Hello")

AIMessage(content='Hello! 👋 \n\nHow can I help you today? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 11, 'total_tokens': 27, 'completion_time': 0.029090909, 'prompt_time': 0.001873167, 'queue_time': 0.15790933199999999, 'total_time': 0.030964076}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f73404c3-d170-404b-b323-232b5a698924-0', usage_metadata={'input_tokens': 11, 'output_tokens': 16, 'total_tokens': 27})

In [13]:
#defining a Pydantic outputparser to format the llm output
class Listing(BaseModel):
    neighborhood: str = Field(..., description="Name of the neighborhood")
    price: int = Field(..., description="Listing price in whole US dollars")
    bedrooms: int = Field(..., description="Number of bedrooms (1-6)")
    bathrooms: int = Field(..., description="Number of bathrooms (1-4)")
    house_size: str = Field(..., description='Living area, e.g. "2150 sqft"')
    description: str = Field(
        ..., description="5–6 engaging sentences describing the property"
    )
    neighborhood_description: str = Field(
        ..., description="3–4 sentences describing the neighborhood"
    )
#defining the parser
#parser = PydanticOutputParser(pydantic_object=Listing)

#binding the llm with the paser
listing_llm = llm_groq.with_structured_output(Listing)

In [17]:
# designing a Prompt template (the parser adds format instructions automatically)
prompt = PromptTemplate(
    template=(
        "You are an expert real-estate copywriter.\n\n"
        "Generate a **fictional but realistic** property listing that follows this brief:\n"
        "• Each call must describe a different neighborhood.\n"
        "• Keep data plausible and coherent.\n\n"
    ),
)

In [18]:
# Example LLM chain
chain = prompt | listing_llm  

# Generate one listing
listing = chain.invoke({})
print(listing)

neighborhood='Willow Creek' price=525000 bedrooms=4 bathrooms=3 house_size='2500 sqft' description='This stunning 4-bedroom, 3-bathroom home in the heart of Willow Creek offers a blend of modern comfort and timeless charm. With its spacious living areas, gourmet kitchen, and private backyard oasis, this home is perfect for families looking for space and tranquility.' neighborhood_description='Willow Creek is a vibrant and family-friendly neighborhood known for its excellent schools, peaceful parks, and charming downtown area.  Residents enjoy a strong sense of community and easy access to shopping, dining, and entertainment.'


In [19]:
# ---------------------------------------------------------------------
# Generate N listings with retry / pacing
# ---------------------------------------------------------------------
def generate_listings(
    n: int,
    chain,                     # the LangChain pipeline (prompt | llm | parser)
    pause: float = 0.3,        # polite delay between calls in seconds
    max_retries: int = 2,      # how many times to retry on an error
) -> List[Listing]:
    """Generate *n* real-estate listings via the Groq chain."""
    
    listings: List[Listing] = []
    
    for i in range(n):
        attempts = 0
        while attempts <= max_retries:
            try:
                listing = chain.invoke({})        # no inputs thanks to partial_variables
                listings.append(listing)
                break                            # success → exit retry loop
            except Exception as e:
                attempts += 1
                if attempts > max_retries:
                    print(f"[{i}] failed after {max_retries} retries → {e}")
                else:
                    print(f"[{i}] error, retrying ({attempts}) → {e}")
                    time.sleep(pause)
        time.sleep(pause)   # pacing to avoid rate limits
    return listings

In [20]:
# ---------------------------------------------------------------------
# Optional helper: save to JSON
# ---------------------------------------------------------------------
def save_listings(
    listings: List[Listing],
    path: str = "./listings.json",
):
    """Serialize Listing objects to pretty-printed JSON."""
    
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as fp:
        json.dump([l.dict() for l in listings], fp, indent=2)
    print(f"✅ Saved {len(listings)} listings → {path}")

data = generate_listings(50, chain)
save_listings(data)

✅ Saved 50 listings → ./listings.json


/tmp/ipykernel_248131/1353327800.py:12: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump([l.dict() for l in listings], fp, indent=2)


In [5]:
#Load the listings
# 2. -------------- Load listings.json -------------------------------------
LISTING_FILE = "./listings.json"

with open(LISTING_FILE, "r") as fp:
    raw_listings = json.load(fp) 

raw_listings[0:4]

[{'neighborhood': 'Oakwood Estates',
  'price': 650000,
  'bedrooms': 4,
  'bathrooms': 3,
  'house_size': '2500 sqft',
  'description': "Welcome to your dream home! This beautifully renovated 4 bedroom, 3 bathroom residence offers modern elegance and spacious living. Enjoy the gourmet kitchen, perfect for entertaining, and relax in the tranquil master suite. The large backyard is an oasis, ideal for summer gatherings. Don't miss this opportunity to own a slice of paradise.",
  'neighborhood_description': 'Oakwood Estates is a peaceful and family-friendly community with tree-lined streets, top-rated schools, and convenient access to parks, shopping, and dining.  Enjoy a sense of community and a tranquil lifestyle in this beautiful neighborhood.'},
 {'neighborhood': 'Oakwood',
  'price': 620000,
  'bedrooms': 4,
  'bathrooms': 3,
  'house_size': '2500 sqft',
  'description': 'This beautifully updated 4 bedroom, 3 bathroom home in Oakwood is move-in ready! The spacious living area featur